# 📏 LiH Ground State Energy vs. Bond Length (Noiseless VQE)

This notebook applies the **Variational Quantum Eigensolver (VQE)** algorithm to explore how the **ground state energy of lithium hydride (LiH)** varies with **bond length**, using PennyLane's quantum chemistry tools in a **noiseless simulation**.

### 🔍 Objective:
To identify the **optimal Li–H bond length** by minimizing the ground state energy through a series of VQE simulations over a range of interatomic distances.

### Key Features:
- **Bond Length Sweep**: Scans 10 evenly spaced Li–H bond lengths from 1.1 Å to 2.1 Å.
- **Dynamic Geometry & Hamiltonian**: Recalculates molecular coordinates and the Hamiltonian at each bond length.
- **UCC Ansatz**: Applies a double-excitation ansatz based on UCCSD theory.
- **VQE Optimization**: Uses **Gradient Descent** to minimize energy for each configuration.
- **Energy Profiling**:
  - Plots ground state energy as a function of bond length.
  - Identifies the bond length that minimizes the energy (i.e., the optimal geometry).

### Outcome:
This notebook estimates the **equilibrium bond distance** of LiH using VQE, serving as a prototype for geometry optimization in quantum computational chemistry.

In [ ]:
import pennylane as qml                  # Quantum circuit builder and device management
from pennylane import numpy as np        # Not regular NumPy, but similar and supports automatic differentiation
from pennylane import qchem              # Quantum chemistry module used to define molecule Hamiltonians
from pennylane.qchem import excitations  # Single and double excitations used in the UCCSD (Unitary Coupled Cluster Singles and Doubles) ansatz
import matplotlib.pyplot as plt
import json, os, time
from vqe_utils import excitation_ansatz, get_optimizer, set_seed, make_run_config_dict, run_signature, find_existing_run, save_run_record, ensure_dirs
from vqe_utils import RUNS_DIR, RESULTS_DIR, IMG_DIR

seed = 0
set_seed(seed)  # Reproducible runs
ensure_dirs()   # Creates runs/, results/ and images, if missing

In [ ]:
# Define the atoms in the LiH molecule and scan settings
symbols = ["Li", "H"]
bond_lengths = np.linspace(1.1, 2.1, 10)
energies = []

# Default STO-3G basis set (explicit for reproducibility)
basis = "STO-3G"

def lih_geometry(bond_length):
    return np.array([
        [0.0, 0.0, 0.0],         # Li
        [0.0, 0.0, bond_length]  # H
    ])


def run_vqe_for_bond_length(bond_length, max_iters=25, stepsize=0.8):
    """
    Runs (or loads) a VQE optimization for LiH at a given bond length using UCC doubles only.
    - If a JSON record exists in runs/ for the exact same config (symbols, geometry, basis,
      ansatz, optimizer name+stepsize, iterations_planned, seed), reuse it.
    - Otherwise compute, then save a JSON run record.
    Returns: (final_energy: float, energy_curve: list[float])
    """
    # Build geometry and Hamiltonian for this bond length
    coordinates = lih_geometry(bond_length)
    hamiltonian, qubits = qchem.molecular_hamiltonian(symbols, coordinates, charge=0, basis=basis)
    electrons = 4
    hf = qchem.hf_state(electrons=electrons, orbitals=qubits)
    _, doubles = excitations(electrons, qubits)

    # UCC doubles only
    ansatz_desc = "UCC doubles (excitation_ansatz with doubles only)"

    # Optimizer config (Gradient Descent for this scan)
    optimizer_name = "GradientDescent"

    # Build configuration and signature (no circuit calls here)
    cfg = make_run_config_dict(
        symbols=symbols,
        coordinates=coordinates,         # includes bond_length; rounded in the helper for stable hashing
        basis=basis,
        ansatz_desc=ansatz_desc,
        optimizer_name=optimizer_name,
        stepsize=stepsize,
        max_iterations=max_iters,
        seed=seed,
    )
    sig = run_signature(cfg)

    # Human-readable filename + signature (include bond length for quick identification)
    fname = os.path.join(RUNS_DIR, f"LiH_BondLen_{bond_length:.3f}A_{optimizer_name}_s{seed}__{sig}.json")
    existing = find_existing_run(RUNS_DIR, sig)

    # Create device (noiseless state vector simulator)
    dev = qml.device("default.qubit", wires=qubits)

    @qml.qnode(dev)
    def cost_function(params):
        excitation_ansatz(params, wires=range(qubits), hf_state=hf,
                          excitations=doubles, excitation_type="double")
        return qml.expval(hamiltonian)

    if existing:
        # Load results if this configuration has been run before
        with open(existing) as f:
            rec = json.load(f)
        energy_curve = rec["energies"]
        final_energy = float(energy_curve[-1])

        # Backfill to results/ if we loaded from runs/
        base = os.path.basename(existing)
        dest = os.path.join(RESULTS_DIR, base)
        if not os.path.exists(dest):
            with open(dest, "w") as f:
                json.dump(rec, f, indent=2)
            print(f"[mirrored-existing] {dest}")

        return final_energy, energy_curve

    # Compute results if this configuration has NOT been ran before
    params = np.zeros(len(doubles), requires_grad=True)
    opt = get_optimizer(optimizer_name, stepsize=stepsize)
    energy_curve = [cost_function(params)]
    for _ in range(max_iters):
        params, e_prev = opt.step_and_cost(cost_function, params)
        energy_curve.append(e_prev)
    energy_curve[-1] = float(cost_function(params))  # Refresh final point
    final_energy = float(energy_curve[-1])

    run_record = {
        "timestamp": time.strftime("%Y-%m-%d %H:%M:%S"),    # Date and time of run
        "molecule": "LiH",
        "symbols": symbols,
        "geometry": coordinates.tolist(),
        "basis": basis,
        "electrons": electrons,
        "num_wires": qubits,
        "ansatz": ansatz_desc,
        "optimizer": {
            "name": optimizer_name,
            "stepsize": stepsize,
            "iterations_planned": max_iters,
            "iterations_ran": len(energy_curve) - 1,
        },
        "seed": seed,
        "energies": [float(e) for e in energy_curve],       # List of energies per iteration
        "final_params": [float(x) for x in params],         # Store only final params
        "config_hash": sig,
        "metadata": {"bond_length_A": float(bond_length)},  # Explicit metadata for convenience
    }

    persisted = save_run_record(fname, run_record)
    print(f"[saved] {optimizer_name}: {fname}")
    print(f"[mirrored] {persisted}")

    return final_energy, energy_curve

energies = []
curves = {}

for r in bond_lengths:
    final_E, E_curve = run_vqe_for_bond_length(r, max_iters=25, stepsize=0.8)
    energies.append(final_E)
    curves[r] = E_curve
    print(f"r = {r:.2f} Å → E = {final_E:.8f} Ha")


In [ ]:
plt.plot(bond_lengths, energies, marker='o')
plt.xlabel("Li–H bond length (Å)")
plt.ylabel("Ground state energy (Ha)")
plt.title("LiH VQE: Energy vs. Bond Length (Noiseless, UCC Doubles)")
plt.grid(True)
plt.tight_layout()
plt.savefig(f"{IMG_DIR}/LiH_Bond_Length_Scan.png", dpi=300)
plt.show()

In [ ]:
energies = np.array(energies)    # Convert energies list to a NumPy array (for easy argmin)
min_index = np.argmin(energies)  # Find index of the minimum energy

# Retrieve optimal bond length and energy
optimal_r = bond_lengths[min_index]
optimal_energy = energies[min_index]

# Print result
print(f"\nOptimal bond length: {optimal_r:.2f} Å")
print(f"Minimum ground state energy: {optimal_energy:.8f} Ha")